In [2]:
#Import libraries
import numpy as np
from numpy import random
import math

#ignore warnings about almost ill-conditioned matrices
import warnings
warnings.filterwarnings('ignore') 

In [21]:
#Generate dataset

n = 25 #datapoints
d = 4 #features
x_true = random.normal(size=(n,d), scale=math.pi*2)
coef = random.normal(size=(6*d), scale=math.pi*2)

def label_computation(arr):
  poly = np.dot(arr, coef[:d]) + np.dot(arr**2, coef[d:2*d]) + np.dot(arr**3, coef[2*d:3*d])
  trig = np.dot(np.sin(arr), coef[3*d:4*d]) + np.dot(np.cos(arr), coef[4*d:5*d])
  exp = np.dot(np.exp(arr), coef[5*d:6*d])
  epsilon = random.normal(scale=100) #random perturbation
  return poly + trig + exp + epsilon

y_labels = np.apply_along_axis(label_computation, 1, x_true)

In [22]:
#Data preprocessing
x_training_data = x_true
y_training_label = y_labels

def non_linear_transformation(arr):
  poly = np.append(np.append(np.append(arr, np.append(arr**2, arr**3)), arr**4), arr**5)
  trig = np.append(np.append(np.append(np.sin(arr), np.cos(arr)), np.append(np.sin(arr) * np.sin(arr), np.cos(arr) * np.cos(arr))), np.cos(arr)*np.sin(arr))
  exp = np.append(np.exp(arr), np.exp(-arr))
  return np.append(poly, np.append(trig, exp))

x_matrix_training_transformed = np.apply_along_axis(non_linear_transformation, axis=1, arr=x_training_data)

In [23]:
#Linear model
from sklearn import linear_model

lin_reg = linear_model.LinearRegression()
lin_reg.fit(x_matrix_training_transformed, y_training_label)

# print("Computed coefs: ")
# print(lin_reg.coef_)
# print()
# print("Actual coefs: ")
# print(coef)

LinearRegression()

In [24]:
#Ridge reg
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1)
ridge_reg.fit(x_matrix_training_transformed, y_training_label)
# print("Computed coefs: ")
# print(ridge_reg.coef_)
# print()
# print("Actual coefs: ")
# print(coef)

Ridge(alpha=1)

In [25]:
#ridge reg model with cross valid
from sklearn.linear_model import RidgeCV
ridge_reg_cv = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10], cv=10).fit(x_matrix_training_transformed, y_training_label)
# print("Computed coefs: ")
# print(ridge_reg_cv.coef_)
# print()
# print("Actual coefs: ")
# print(coef)


In [26]:
#Generate test data
x_test = random.normal(size=(n,d), scale=math.pi*10)
x_matrix_test_transformed = np.apply_along_axis(non_linear_transformation, axis=1, arr=x_test)

def true_label_computation(arr):
  poly = np.dot(arr, coef[:d]) + np.dot(arr**2, coef[d:2*d]) + np.dot(arr**3, coef[2*d:3*d])
  trig = np.dot(np.sin(arr), coef[3*d:4*d]) + np.dot(np.cos(arr), coef[4*d:5*d])
  exp = np.dot(np.exp(arr), coef[5*d:6*d])
  return poly + trig + exp

y_test = np.apply_along_axis(true_label_computation, 1, x_test)

# print(y_test)
# print()
# print(lin_reg.predict(x_matrix_test_transformed))
# print()
# print(ridge_reg.predict(x_matrix_test_transformed))
# print()
# print(ridge_reg_cv.predict(x_matrix_test_transformed))
print("Linear regression score: " + str(lin_reg.score(x_matrix_test_transformed, y_test)))
print("Ridge regression (without CV) score: " + str(ridge_reg.score(x_matrix_test_transformed, y_test)))
print("Ridge regression with CV score: " + str(ridge_reg_cv.score(x_matrix_test_transformed, y_test)))

Linear regression score: 0.9985996921714243
Ridge regression (without CV) score: 0.9986185130398241
Ridge regression with CV score: 0.9986793599943817
